In [2]:
from utils.data_scrap_functions import *

In [12]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


Ce notebook est temporaire, il sera remplacé par un moyen plus simple et ergonomique d'importer les données.

## Importation des données

### Dans un premier temps, on importe la liste de toute les variables à récupérer

In [3]:
url = 'https://classic.clinicaltrials.gov/api/info/study_fields_list'

In [4]:
list_fields = get_list(url)

In [5]:
list_fields[:20]

['Acronym',
 'AgreementOtherDetails',
 'AgreementPISponsorEmployee',
 'AgreementRestrictionType',
 'AgreementRestrictiveAgreement',
 'ArmGroupDescription',
 'ArmGroupInterventionName',
 'ArmGroupLabel',
 'ArmGroupType',
 'AvailIPDComment',
 'AvailIPDId',
 'AvailIPDType',
 'AvailIPDURL',
 'BaselineCategoryTitle',
 'BaselineClassDenomCountGroupId',
 'BaselineClassDenomCountValue',
 'BaselineClassDenomUnits',
 'BaselineClassTitle',
 'BaselineDenomCountGroupId',
 'BaselineDenomCountValue']

### Maintenant, on importe les données depuis l'API de Clinical Trials selon la liste de variable qu'on veut

In [7]:
get_data(taille=5,list_fields=list_fields[-40:])

100%|██████████| 10/10 [00:59<00:00,  5.98s/it]

Temps requete API: -------------------- 59.55615782737732 

 Temps Pandas: -------------------- 0.19400429725646973



/home/onyxia/work/ProjetPython/utils/data_scrap_functions.py:95: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  full_df = full_df.applymap(treat_list)


,RetractionSource,SamplingMethod,SecondaryId,SecondaryIdDomain,SecondaryIdLink,SecondaryIdType,SecondaryOutcomeDescription,SecondaryOutcomeMeasure,SecondaryOutcomeTimeFrame,SeeAlsoLinkLabel,...,SubmissionMCPReleaseN,SubmissionReleaseDate,SubmissionResetDate,SubmissionUnreleaseDate,TargetDuration,UnpostedEventDate,UnpostedEventType,UnpostedResponsibleParty,VersionHolder,WhyStopped
0,None,Probability Sample,None,None,None,None,None,None,None,None,...,None,None,None,None,6 Months,None,None,None,"December 15, 2023",None
1,None,None,None,None,None,None,None,Cmax of Itraconazole,Day 4 up to Day 6,None,...,None,None,None,None,None,None,None,None,"December 15, 2023",None
2,None,Non-Probability Sample,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,"December 15, 2023",None
3,None,None,None,None,None,None,Blood samples will be collected and LDL-C leve...,Percent Change from Baseline of Low-Density Li...,From first day of treatment up to week 12.,None,...,None,None,None,None,None,None,None,None,"December 15, 2023",None
4,None,Non-Probability Sample,0025-15-ziv,ZivMC,None,Registry Identifier,None,None,None,None,...,None,None,None,None,2 Years,None,None,None,"December 15, 2023",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,None,None,None,None,None,None,Analysis of the primary endpoint in relation w...,.Analysis of the primary endpoint related to A...,Day 4,None,...,None,None,None,None,None,None,None,None,"December 15, 2023",None
9996,None,None,None,None,None,None,Body weight will be measured on a beam balance...,Body weight,Baseline,None,...,None,None,None,None,None,None,None,None,"December 15, 2023",None
9997,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,"December 15, 2023",None
9998,None,None,None,None,None,None,Disease control rate: The proportion of subjec...,Disease control rate (DCR) within 3 months aft...,3 months,None,...,None,None,None,None,None,None,None,None,"December 15, 2023",None


In [8]:
df = get_data_parralel(taille=100,list_fields=list_fields)
df

  7%|▋         | 7/100 [00:45<09:50,  6.35s/it]

Erreur de connexion à l'url https://classic.clinicaltrials.gov/api/query/study_fields?expr=&fields=OutcomeMeasureParamType%2COutcomeMeasurePopulationDescription%2COutcomeMeasureReportingStatus%2COutcomeMeasureTimeFrame%2COutcomeMeasureTitle%2COutcomeMeasureType%2COutcomeMeasureTypeUnitsAnalyzed%2COutcomeMeasureUnitOfMeasure%2COutcomeMeasurementComment%2COutcomeMeasurementGroupId%2COutcomeMeasurementLowerLimit%2COutcomeMeasurementSpread%2COutcomeMeasurementUpperLimit%2COutcomeMeasurementValue%2COverallOfficialAffiliation%2COverallOfficialName%2COverallOfficialRole%2COverallStatus%2COversightHasDMC%2CPatientRegistry&min_rnk=7001&max_rnk=8000&fmt=json: 503 Server Error: Service Unavailable for url: https://classic.clinicaltrials.gov/api/query/study_fields?expr=&fields=OutcomeMeasureParamType%2COutcomeMeasurePopulationDescription%2COutcomeMeasureReportingStatus%2COutcomeMeasureTimeFrame%2COutcomeMeasureTitle%2COutcomeMeasureType%2COutcomeMeasureTypeUnitsAnalyzed%2COutcomeMeasureUnitOfMeasu

  9%|▉         | 9/100 [01:10<14:01,  9.25s/it]

Erreur de connexion à l'url https://classic.clinicaltrials.gov/api/query/study_fields?expr=&fields=SubmissionResetDate%2CSubmissionUnreleaseDate%2CTargetDuration%2CUnpostedEventDate%2CUnpostedEventType%2CUnpostedResponsibleParty%2CVersionHolder%2CWhyStopped&min_rnk=9001&max_rnk=10000&fmt=json: 500 Server Error: Internal Server Error for url: https://classic.clinicaltrials.gov/api/query/study_fields?expr=&fields=SubmissionResetDate%2CSubmissionUnreleaseDate%2CTargetDuration%2CUnpostedEventDate%2CUnpostedEventType%2CUnpostedResponsibleParty%2CVersionHolder%2CWhyStopped&min_rnk=9001&max_rnk=10000&fmt=json


 11%|█         | 11/100 [01:51<22:51, 15.41s/it]

...

 26%|██▌       | 26/100 [04:15<07:22,  5.98s/it]

Erreur de connexion à l'url https://classic.clinicaltrials.gov/api/query/study_fields?expr=&fields=Phase%2CPointOfContactEMail%2CPointOfContactOrganization%2CPointOfContactPhone%2CPointOfContactPhoneExt%2CPointOfContactTitle%2CPrimaryCompletionDate%2CPrimaryCompletionDateType%2CPrimaryOutcomeDescription%2CPrimaryOutcomeMeasure%2CPrimaryOutcomeTimeFrame%2CReferenceCitation%2CReferencePMID%2CReferenceType%2CRemovedCountry%2CResponsiblePartyInvestigatorAffiliation%2CResponsiblePartyInvestigatorFullName%2CResponsiblePartyInvestigatorTitle%2CResponsiblePartyOldNameTitle%2CResponsiblePartyOldOrganization&min_rnk=26001&max_rnk=27000&fmt=json: 503 Server Error: Service Unavailable for url: https://classic.clinicaltrials.gov/api/query/study_fields?expr=&fields=Phase%2CPointOfContactEMail%2CPointOfContactOrganization%2CPointOfContactPhone%2CPointOfContactPhoneExt%2CPointOfContactTitle%2CPrimaryCompletionDate%2CPrimaryCompletionDateType%2CPrimaryOutcomeDescription%2CPrimaryOutcomeMeasure%2CPrimar

100%|██████████| 100/100 [11:41<00:00,  7.01s/it]
/home/onyxia/work/ProjetPython/utils/data_scrap_functions.py:155: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  full_df = full_df.applymap(treat_list)


Temps requete API: -------------------- 647.3866486549377 

 Temps Pandas: -------------------- 53.65038800239563


,Acronym,AgreementOtherDetails,AgreementPISponsorEmployee,AgreementRestrictionType,AgreementRestrictiveAgreement,ArmGroupDescription,ArmGroupInterventionName,ArmGroupLabel,ArmGroupType,AvailIPDComment,...,SubmissionMCPReleaseN,SubmissionReleaseDate,SubmissionResetDate,SubmissionUnreleaseDate,TargetDuration,UnpostedEventDate,UnpostedEventType,UnpostedResponsibleParty,VersionHolder,WhyStopped
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,6 Months,None,None,None,"December 15, 2023",None
1,None,None,None,None,None,"During Part 1, Period 1, participants will rec...",Drug: ALXN2080,Part 1,Experimental,None,...,None,None,None,None,None,None,None,None,"December 15, 2023",None
2,None,None,None,None,None,Adult patients undergoing elective open-heart ...,Diagnostic Test: Detection/screening,Adult In-patient Cardiac surgery,None,None,...,None,None,None,None,None,None,None,None,"December 15, 2023",None
3,PURSUIT,None,None,None,None,"AZD0780, Dose 1",Drug: AZD0780,Arm A,Experimental,None,...,None,None,None,None,None,None,None,None,"December 15, 2023",None
4,THR,None,None,None,None,None,None,None,None,None,...,None,None,None,None,2 Years,None,None,None,"December 15, 2023",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,None,None,None,None,None,Subjects received gong's mobilization three ti...,Other: Gong's Mobilization,Gong's Mobilization,Experimental,None,...,None,None,None,None,None,None,None,None,"December 15, 2023",None
99996,None,None,None,None,None,Participants undergoing foot or ankle surgery ...,Drug: Ropivacaine 0.5% Injectable Solution,Standard Popliteal Nerve Block,Active Comparator,None,...,None,None,None,None,None,None,None,None,"December 15, 2023",None
99997,MURDOCH,None,None,None,None,Adults with physician diagnosis of stable asth...,None,Asthma Patients,None,None,...,None,None,None,None,None,None,None,None,"December 15, 2023",None
99998,None,None,None,None,None,Participants treated for an ankle fracture wil...,Device: Controlled Ankle Motion (CAM) boot,Early weight bearing,Active Comparator,None,...,None,None,None,None,None,None,None,None,"December 15, 2023",None


In [9]:
df.to_csv('./data/clini_data.csv')